In [1]:
import wave

In [2]:
import struct

In [3]:
import sys

In [4]:
def bitleriDamgala(damgaBoyutu, nBit = 8):
    damgaBitleri = []
    for byte in damgaBoyutu:
        for i in range(0, nBit):
            damgaBitleri.append((byte & (2 ** i)) >> i)
    return damgaBitleri

In [5]:
def lsbDamgala(ses, mesaj, cikti):
    varsayilanBit = 32
    mesaj = str(mesaj)
    damga = struct.unpack("%dB" % len(mesaj), mesaj)
    damgaBoyutu = len(damga)
    damgaBitleri = bitleriDamgala((damgaBoyutu, ), varsayilanBit)
    damgaBitleri.extend(bitleriDamgala(damga))
    sesFiltresi = wave.open(ses, 'rb')
    (kanallar, kesitUzunlugu, cerceveOrani, nCerceve, tip, isim) = sesFiltresi.getparams()
    cerceveler = sesFiltresi.readframes(nCerceve * kanallar)
    kesitler = struct.unpack_from("%dh" % nCerceve * kanallar, cerceveler)
    if len(kesitler) < len(damgaBitleri):
        print("Damga verisi kesitten daha büyük olamaz.")
    sifreliKesitler = []
    damgaKonumu = 0
    for kesit in kesitler:
        sifreliKesitler = kesit
        if damgaKonumu < len(damgaBitleri):
            sifreliBit = damgaBitleri[damgaKonumu]
            if sifreliBit == 1:
                sifreliKesit = kesit | sifreliBit
            else:
                sifreliKesit = kesit
                if kesit & 1 != 0:
                    sifreliKesit = kesit - 1
            damgaKonumu += 1
        sifreliKesitler.append(sifreliKesit)
    sifreliSes = wave.open(cikti, 'wb')
    sifreliSes.setparams((kanallar, kesitUzunlugu, cerceveOrani, nCerceve, tip, isim))
    sifreliSes.writeframes(struct.pack("%dh" % len(sifreliKesitler), *sifreliKesitler))

In [6]:
def lsbDesifrele(cikti):
    varsayilanBit = 32
    damgaliSes = wave.open(cikti, 'rb')
    (kanallar, kesitUzunlugu, cerceveOrani, nCerceve, tip, isim) = damgaliSes.getparams()
    cerceveler = damgaliSes.readframes(nCerceve * kanallar)
    kesitler = struct.unpack_from("%dh" % nCerceve * kanallar, cerceveler)
    damgaByte = 0
    for (kesit, i) in zip(kesitler[0:varsayilanBit], range(0, varsayilanBit)):
        damgaByte += (kesit & 1) * (2 ** i)
    damgaVerisi = []
    for n in range(0, damgaByte):
        damgaByteKesitleri = kesitler[varsayilanBit + (n * 8) : varsayilanBit + ((n + 1) * 8)]
        damgaByte = 0
        for (kesit, i) in zip(damgaByte, range(0, 8)):
            damgaByte += (kesit & 1) * (2 ** i)
        damgaVerisi.append(damgaByte)
    return damgaVerisi

In [7]:
def damgaStr(liste):
    return "".join([chr(x) for x in liste])

In [8]:
def gomuDosyasi(ses, gizliDosya, cikti):
    dosya = open(gizliDosya)
    gizliDosya = dosya.read()
    lsbDamgala(ses, gizliDosya, cikti)

In [9]:
def desifreDosyasi(sifreliSinyal, gizliVeriKonumu):
    damga = lsbDesifrele(sifreliSinyal)
    damgaStr = damgaStr(damga)
    dosya = open(gizliVeriKonumu, "W")
    dosya.write(damgaStr)

In [10]:
ses = ".../sesler/ses.mp3"
output = "x.wav"
if len(sys.argv) > 1:
    mesaj = sys.argv[1]
    if len(sys.argv) > 2:
        ses = sys.argv[2]
        if len(sys.argv) > 3:
            cikti = sys.argv[3]

In [ ]:
lsbDamgala(ses, mesaj, cikti)
lsbDesifrele(cikti)